# Graph Coloring Problem - Optimization with Genetic Algorithms

## Neural and Evolutionary Computation (NEC) - A2

This notebook implements:
1. **Genetic Algorithm (GA)** - Main implementation
2. **Simulated Annealing (SA)** - Additional optimization method
3. **Tabu Search (TS)** - Additional optimization method

### Graph Coloring Problem (GCP)

Given a graph with V vertices and E edges, assign a color to each vertex such that:
- No two adjacent vertices have the same color
- The number of colors used is minimized (chromatic number)

## 1. Imports and Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import seaborn as sns
from typing import List, Tuple, Set, Dict
from collections import defaultdict
import time
import copy
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 2. Graph Data Structure and Utilities

In [ ]:
class Graph:
    """Graph representation for the coloring problem"""
    
    def __init__(self, num_vertices: int):
        self.num_vertices = num_vertices
        self.edges = []
        self.adjacency_list = [set() for _ in range(num_vertices)]
        self.adjacency_matrix = np.zeros((num_vertices, num_vertices), dtype=int)
    
    def add_edge(self, u: int, v: int):
        """Add an edge between vertices u and v (0-indexed)"""
        if u >= self.num_vertices or v >= self.num_vertices:
            raise ValueError(f"Vertex index out of range: {u}, {v}")
        
        self.edges.append((u, v))
        self.adjacency_list[u].add(v)
        self.adjacency_list[v].add(u)
        self.adjacency_matrix[u][v] = 1
        self.adjacency_matrix[v][u] = 1
    
    def get_neighbors(self, vertex: int) -> Set[int]:
        """Get all neighbors of a vertex"""
        return self.adjacency_list[vertex]
    
    def degree(self, vertex: int) -> int:
        """Get the degree of a vertex"""
        return len(self.adjacency_list[vertex])
    
    def max_degree(self) -> int:
        """Get the maximum degree in the graph"""
        return max(self.degree(v) for v in range(self.num_vertices))
    
    def chromatic_upper_bound(self) -> int:
        """Upper bound for chromatic number (max_degree + 1)"""
        return self.max_degree() + 1
    
    def to_networkx(self) -> nx.Graph:
        """Convert to NetworkX graph for visualization"""
        G = nx.Graph()
        G.add_nodes_from(range(self.num_vertices))
        G.add_edges_from(self.edges)
        return G
    
    def __str__(self):
        return f"Graph(vertices={self.num_vertices}, edges={len(self.edges)})"


def load_dimacs_graph(filename: str) -> Graph:
    """
    Load graph from DIMACS format file
    
    Format:
    - c <comment>
    - p edge <num_vertices> <num_edges>
    - e <vertex1> <vertex2>
    """
    graph = None
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            
            if not line or line.startswith('c'):
                continue
            
            parts = line.split()
            
            if parts[0] == 'p':
                num_vertices = int(parts[2])
                graph = Graph(num_vertices)
            
            elif parts[0] == 'e':
                if graph is None:
                    raise ValueError("Edge before problem definition")
                # DIMACS uses 1-indexed vertices
                u = int(parts[1]) - 1
                v = int(parts[2]) - 1
                graph.add_edge(u, v)
    
    return graph


def validate_coloring(graph: Graph, coloring: List[int]) -> Tuple[bool, int]:
    """
    Validate coloring and count conflicts
    
    Returns:
        (is_valid, num_conflicts)
    """
    conflicts = 0
    for u, v in graph.edges:
        if coloring[u] == coloring[v]:
            conflicts += 1
    return (conflicts == 0, conflicts)


def count_colors(coloring: List[int]) -> int:
    """Count number of distinct colors used"""
    return len(set(coloring))


def greedy_coloring(graph: Graph) -> List[int]:
    """Simple greedy coloring algorithm"""
    coloring = [-1] * graph.num_vertices
    
    for vertex in range(graph.num_vertices):
        neighbor_colors = {coloring[n] for n in graph.get_neighbors(vertex) if coloring[n] != -1}
        color = 0
        while color in neighbor_colors:
            color += 1
        coloring[vertex] = color
    
    return coloring


def dsatur_coloring(graph: Graph) -> List[int]:
    """
    DSATUR (Degree of Saturation) coloring algorithm
    Often produces better initial solutions
    """
    coloring = [-1] * graph.num_vertices
    uncolored = set(range(graph.num_vertices))
    
    while uncolored:
        max_saturation = -1
        max_degree = -1
        next_vertex = None
        
        for vertex in uncolored:
            neighbor_colors = {coloring[n] for n in graph.get_neighbors(vertex) if coloring[n] != -1}
            saturation = len(neighbor_colors)
            
            if saturation > max_saturation or (saturation == max_saturation and graph.degree(vertex) > max_degree):
                max_saturation = saturation
                max_degree = graph.degree(vertex)
                next_vertex = vertex
        
        neighbor_colors = {coloring[n] for n in graph.get_neighbors(next_vertex) if coloring[n] != -1}
        color = 0
        while color in neighbor_colors:
            color += 1
        
        coloring[next_vertex] = color
        uncolored.remove(next_vertex)
    
    return coloring

## 3. Visualization Functions

In [ ]:
def visualize_graph_coloring(graph: Graph, coloring: List[int], title: str = "Graph Coloring"):
    """
    Visualize a colored graph
    """
    G = graph.to_networkx()
    
    # Create color map
    num_colors = len(set(coloring))
    colors = plt.cm.rainbow(np.linspace(0, 1, num_colors))
    color_map = [colors[coloring[node]] for node in G.nodes()]
    
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, seed=42)
    
    nx.draw(G, pos, node_color=color_map, with_labels=True, 
            node_size=500, font_size=10, font_weight='bold',
            edge_color='gray', width=1.5)
    
    is_valid, conflicts = validate_coloring(graph, coloring)
    num_colors_used = count_colors(coloring)
    
    plt.title(f"{title}\nColors: {num_colors_used}, Valid: {is_valid}, Conflicts: {conflicts}")
    plt.tight_layout()
    plt.show()


def plot_fitness_evolution(fitness_history: List[float], title: str = "Fitness Evolution"):
    """
    Plot the evolution of fitness over generations
    """
    plt.figure(figsize=(12, 6))
    plt.plot(fitness_history, linewidth=2)
    plt.xlabel('Generation', fontsize=12)
    plt.ylabel('Best Fitness (lower is better)', fontsize=12)
    plt.title(title, fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_algorithm_comparison(results: Dict[str, Dict]):
    """
    Compare different algorithms
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    algorithms = list(results.keys())
    colors_used = [results[alg]['colors'] for alg in algorithms]
    execution_time = [results[alg]['time'] for alg in algorithms]
    iterations = [results[alg]['iterations'] for alg in algorithms]
    
    # Colors used
    axes[0].bar(algorithms, colors_used, color='steelblue')
    axes[0].set_ylabel('Number of Colors')
    axes[0].set_title('Colors Used (lower is better)')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Execution time
    axes[1].bar(algorithms, execution_time, color='coral')
    axes[1].set_ylabel('Time (seconds)')
    axes[1].set_title('Execution Time')
    axes[1].tick_params(axis='x', rotation=45)
    
    # Iterations
    axes[2].bar(algorithms, iterations, color='lightgreen')
    axes[2].set_ylabel('Iterations')
    axes[2].set_title('Iterations to Convergence')
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## 4. Genetic Algorithm Implementation

### 4.1 Chromosome Representation

**Chromosome encoding**: A chromosome is represented as a list of integers where:
- `chromosome[i]` = color assigned to vertex `i`
- Colors are integers from 0 to k-1 (where k is the number of colors)

**Example**: For a 5-vertex graph, `[0, 1, 0, 2, 1]` means:
- Vertex 0 → Color 0
- Vertex 1 → Color 1
- Vertex 2 → Color 0
- Vertex 3 → Color 2
- Vertex 4 → Color 1

In [ ]:
class GeneticAlgorithm:
    """
    Genetic Algorithm for Graph Coloring Problem
    """
    
    def __init__(self, graph: Graph, population_size: int = 100,
                 mutation_rate: float = 0.1, crossover_rate: float = 0.8,
                 elitism_size: int = 2, max_generations: int = 1000,
                 selection_method: str = 'tournament',
                 crossover_method: str = 'uniform',
                 mutation_method: str = 'random'):
        """
        Initialize Genetic Algorithm
        
        Args:
            graph: Graph to color
            population_size: Number of individuals in population
            mutation_rate: Probability of mutation
            crossover_rate: Probability of crossover
            elitism_size: Number of best individuals to keep
            max_generations: Maximum number of generations
            selection_method: 'tournament' or 'roulette'
            crossover_method: 'uniform' or 'single_point'
            mutation_method: 'random' or 'smart'
        """
        self.graph = graph
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elitism_size = elitism_size
        self.max_generations = max_generations
        self.selection_method = selection_method
        self.crossover_method = crossover_method
        self.mutation_method = mutation_method
        
        self.max_colors = graph.chromatic_upper_bound()
        self.population = []
        self.fitness_history = []
        self.best_solution = None
        self.best_fitness = float('inf')
    
    def fitness(self, chromosome: List[int]) -> float:
        """
        Fitness function: minimize (conflicts * 1000 + number_of_colors)
        
        This prioritizes valid solutions (0 conflicts) and then minimizes colors
        """
        is_valid, conflicts = validate_coloring(self.graph, chromosome)
        num_colors = count_colors(chromosome)
        
        # Heavy penalty for conflicts, then minimize colors
        return conflicts * 1000 + num_colors
    
    def initialize_population(self):
        """Initialize population with mix of random and greedy solutions"""
        self.population = []
        
        # Add greedy solution
        greedy_sol = greedy_coloring(self.graph)
        self.population.append(greedy_sol)
        
        # Add DSATUR solution
        dsatur_sol = dsatur_coloring(self.graph)
        self.population.append(dsatur_sol)
        
        # Fill rest with random solutions
        for _ in range(self.population_size - 2):
            chromosome = [np.random.randint(0, self.max_colors) 
                         for _ in range(self.graph.num_vertices)]
            self.population.append(chromosome)
    
    def tournament_selection(self, tournament_size: int = 5) -> List[int]:
        """Tournament selection: select best from random sample"""
        tournament = np.random.choice(len(self.population), tournament_size, replace=False)
        best_idx = min(tournament, key=lambda i: self.fitness(self.population[i]))
        return self.population[best_idx].copy()
    
    def roulette_selection(self) -> List[int]:
        """Roulette wheel selection: probability proportional to inverse fitness"""
        fitnesses = np.array([self.fitness(ind) for ind in self.population])
        # Invert fitness (lower is better)
        max_fitness = fitnesses.max()
        inverted_fitness = max_fitness - fitnesses + 1
        probabilities = inverted_fitness / inverted_fitness.sum()
        
        selected_idx = np.random.choice(len(self.population), p=probabilities)
        return self.population[selected_idx].copy()
    
    def select_parent(self) -> List[int]:
        """Select parent based on configured method"""
        if self.selection_method == 'tournament':
            return self.tournament_selection()
        else:  # roulette
            return self.roulette_selection()
    
    def uniform_crossover(self, parent1: List[int], parent2: List[int]) -> Tuple[List[int], List[int]]:
        """Uniform crossover: each gene randomly from either parent"""
        child1 = []
        child2 = []
        
        for i in range(len(parent1)):
            if np.random.random() < 0.5:
                child1.append(parent1[i])
                child2.append(parent2[i])
            else:
                child1.append(parent2[i])
                child2.append(parent1[i])
        
        return child1, child2
    
    def single_point_crossover(self, parent1: List[int], parent2: List[int]) -> Tuple[List[int], List[int]]:
        """Single point crossover: split at random point"""
        point = np.random.randint(1, len(parent1))
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    
    def crossover(self, parent1: List[int], parent2: List[int]) -> Tuple[List[int], List[int]]:
        """Crossover based on configured method"""
        if np.random.random() < self.crossover_rate:
            if self.crossover_method == 'uniform':
                return self.uniform_crossover(parent1, parent2)
            else:  # single_point
                return self.single_point_crossover(parent1, parent2)
        else:
            return parent1.copy(), parent2.copy()
    
    def random_mutation(self, chromosome: List[int]) -> List[int]:
        """Random mutation: randomly change colors"""
        mutated = chromosome.copy()
        for i in range(len(mutated)):
            if np.random.random() < self.mutation_rate:
                mutated[i] = np.random.randint(0, self.max_colors)
        return mutated
    
    def smart_mutation(self, chromosome: List[int]) -> List[int]:
        """
        Smart mutation: fix conflicting vertices by changing to valid color
        """
        mutated = chromosome.copy()
        
        # Find conflicting vertices
        conflicts = set()
        for u, v in self.graph.edges:
            if mutated[u] == mutated[v]:
                conflicts.add(u)
                conflicts.add(v)
        
        # Try to fix some conflicts
        for vertex in conflicts:
            if np.random.random() < self.mutation_rate:
                # Get neighbor colors
                neighbor_colors = {mutated[n] for n in self.graph.get_neighbors(vertex)}
                # Try to find a valid color
                available_colors = [c for c in range(self.max_colors) if c not in neighbor_colors]
                if available_colors:
                    mutated[vertex] = np.random.choice(available_colors)
                else:
                    mutated[vertex] = np.random.randint(0, self.max_colors)
        
        return mutated
    
    def mutate(self, chromosome: List[int]) -> List[int]:
        """Mutate based on configured method"""
        if self.mutation_method == 'smart':
            return self.smart_mutation(chromosome)
        else:  # random
            return self.random_mutation(chromosome)
    
    def evolve(self, verbose: bool = True) -> Tuple[List[int], List[float]]:
        """
        Run the genetic algorithm
        
        Returns:
            (best_solution, fitness_history)
        """
        self.initialize_population()
        self.fitness_history = []
        
        generations_without_improvement = 0
        convergence_threshold = 50  # Stop if no improvement for 50 generations
        
        iterator = tqdm(range(self.max_generations)) if verbose else range(self.max_generations)
        
        for generation in iterator:
            # Evaluate fitness
            fitness_values = [(self.fitness(ind), ind) for ind in self.population]
            fitness_values.sort(key=lambda x: x[0])
            
            current_best_fitness = fitness_values[0][0]
            current_best_solution = fitness_values[0][1]
            
            # Track best solution
            if current_best_fitness < self.best_fitness:
                self.best_fitness = current_best_fitness
                self.best_solution = current_best_solution.copy()
                generations_without_improvement = 0
            else:
                generations_without_improvement += 1
            
            self.fitness_history.append(current_best_fitness)
            
            if verbose and generation % 100 == 0:
                is_valid, conflicts = validate_coloring(self.graph, current_best_solution)
                num_colors = count_colors(current_best_solution)
                print(f"Gen {generation}: Fitness={current_best_fitness:.2f}, Colors={num_colors}, Conflicts={conflicts}, Valid={is_valid}")
            
            # Check convergence
            if generations_without_improvement >= convergence_threshold:
                if verbose:
                    print(f"Converged at generation {generation}")
                break
            
            # Create new population
            new_population = []
            
            # Elitism: keep best individuals
            for i in range(self.elitism_size):
                new_population.append(fitness_values[i][1].copy())
            
            # Generate offspring
            while len(new_population) < self.population_size:
                parent1 = self.select_parent()
                parent2 = self.select_parent()
                
                child1, child2 = self.crossover(parent1, parent2)
                
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                
                new_population.append(child1)
                if len(new_population) < self.population_size:
                    new_population.append(child2)
            
            self.population = new_population
        
        return self.best_solution, self.fitness_history

## 5. Simulated Annealing Implementation

In [ ]:
class SimulatedAnnealing:
    """
    Simulated Annealing for Graph Coloring Problem
    """
    
    def __init__(self, graph: Graph, initial_temp: float = 100.0,
                 cooling_rate: float = 0.95, max_iterations: int = 10000):
        self.graph = graph
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        self.max_colors = graph.chromatic_upper_bound()
        self.fitness_history = []
    
    def fitness(self, solution: List[int]) -> float:
        """Fitness function: same as GA"""
        is_valid, conflicts = validate_coloring(self.graph, solution)
        num_colors = count_colors(solution)
        return conflicts * 1000 + num_colors
    
    def get_neighbor(self, solution: List[int]) -> List[int]:
        """Generate neighbor by changing one vertex color"""
        neighbor = solution.copy()
        vertex = np.random.randint(0, len(solution))
        neighbor[vertex] = np.random.randint(0, self.max_colors)
        return neighbor
    
    def optimize(self, verbose: bool = True) -> Tuple[List[int], List[float]]:
        """Run simulated annealing"""
        # Start with greedy solution
        current_solution = dsatur_coloring(self.graph)
        current_fitness = self.fitness(current_solution)
        
        best_solution = current_solution.copy()
        best_fitness = current_fitness
        
        temperature = self.initial_temp
        self.fitness_history = []
        
        iterations_without_improvement = 0
        convergence_threshold = 500
        
        iterator = tqdm(range(self.max_iterations)) if verbose else range(self.max_iterations)
        
        for iteration in iterator:
            # Generate neighbor
            neighbor = self.get_neighbor(current_solution)
            neighbor_fitness = self.fitness(neighbor)
            
            # Acceptance criterion
            delta = neighbor_fitness - current_fitness
            
            if delta < 0 or np.random.random() < np.exp(-delta / temperature):
                current_solution = neighbor
                current_fitness = neighbor_fitness
            
            # Update best
            if current_fitness < best_fitness:
                best_solution = current_solution.copy()
                best_fitness = current_fitness
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
            
            self.fitness_history.append(best_fitness)
            
            # Cool down
            temperature *= self.cooling_rate
            
            if verbose and iteration % 1000 == 0:
                is_valid, conflicts = validate_coloring(self.graph, best_solution)
                num_colors = count_colors(best_solution)
                print(f"Iter {iteration}: Temp={temperature:.2f}, Fitness={best_fitness:.2f}, Colors={num_colors}, Conflicts={conflicts}")
            
            # Check convergence
            if iterations_without_improvement >= convergence_threshold:
                if verbose:
                    print(f"Converged at iteration {iteration}")
                break
        
        return best_solution, self.fitness_history

## 6. Tabu Search Implementation

In [ ]:
class TabuSearch:
    """
    Tabu Search for Graph Coloring Problem
    """
    
    def __init__(self, graph: Graph, tabu_tenure: int = 10,
                 max_iterations: int = 10000):
        self.graph = graph
        self.tabu_tenure = tabu_tenure
        self.max_iterations = max_iterations
        self.max_colors = graph.chromatic_upper_bound()
        self.fitness_history = []
        self.tabu_list = []
    
    def fitness(self, solution: List[int]) -> float:
        """Fitness function: same as GA"""
        is_valid, conflicts = validate_coloring(self.graph, solution)
        num_colors = count_colors(solution)
        return conflicts * 1000 + num_colors
    
    def get_neighbors(self, solution: List[int], num_neighbors: int = 20) -> List[List[int]]:
        """Generate multiple neighbors"""
        neighbors = []
        for _ in range(num_neighbors):
            neighbor = solution.copy()
            vertex = np.random.randint(0, len(solution))
            neighbor[vertex] = np.random.randint(0, self.max_colors)
            neighbors.append(neighbor)
        return neighbors
    
    def is_tabu(self, move) -> bool:
        """Check if move is in tabu list"""
        return move in self.tabu_list
    
    def add_to_tabu(self, move):
        """Add move to tabu list"""
        self.tabu_list.append(move)
        if len(self.tabu_list) > self.tabu_tenure:
            self.tabu_list.pop(0)
    
    def optimize(self, verbose: bool = True) -> Tuple[List[int], List[float]]:
        """Run tabu search"""
        # Start with greedy solution
        current_solution = dsatur_coloring(self.graph)
        current_fitness = self.fitness(current_solution)
        
        best_solution = current_solution.copy()
        best_fitness = current_fitness
        
        self.fitness_history = []
        self.tabu_list = []
        
        iterations_without_improvement = 0
        convergence_threshold = 500
        
        iterator = tqdm(range(self.max_iterations)) if verbose else range(self.max_iterations)
        
        for iteration in iterator:
            # Generate neighbors
            neighbors = self.get_neighbors(current_solution)
            
            # Find best non-tabu neighbor (or best if it improves global best)
            best_neighbor = None
            best_neighbor_fitness = float('inf')
            best_neighbor_move = None
            
            for neighbor in neighbors:
                neighbor_fitness = self.fitness(neighbor)
                
                # Create move representation (simplified)
                move = tuple(neighbor)
                
                # Accept if not tabu or if it's better than global best (aspiration criterion)
                if not self.is_tabu(move) or neighbor_fitness < best_fitness:
                    if neighbor_fitness < best_neighbor_fitness:
                        best_neighbor = neighbor
                        best_neighbor_fitness = neighbor_fitness
                        best_neighbor_move = move
            
            if best_neighbor is not None:
                current_solution = best_neighbor
                current_fitness = best_neighbor_fitness
                self.add_to_tabu(best_neighbor_move)
            
            # Update best
            if current_fitness < best_fitness:
                best_solution = current_solution.copy()
                best_fitness = current_fitness
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
            
            self.fitness_history.append(best_fitness)
            
            if verbose and iteration % 1000 == 0:
                is_valid, conflicts = validate_coloring(self.graph, best_solution)
                num_colors = count_colors(best_solution)
                print(f"Iter {iteration}: Fitness={best_fitness:.2f}, Colors={num_colors}, Conflicts={conflicts}")
            
            # Check convergence
            if iterations_without_improvement >= convergence_threshold:
                if verbose:
                    print(f"Converged at iteration {iteration}")
                break
        
        return best_solution, self.fitness_history

## 7. Download Test Datasets

In [ ]:
import urllib.request
import os

# Create datasets directory
os.makedirs('datasets', exist_ok=True)

# Dataset URLs from https://mat.tepper.cmu.edu/COLOR/instances.html
datasets = {
    'myciel3': 'https://mat.tepper.cmu.edu/COLOR/instances/myciel3.col',  # 11 vertices - small
    'queen5_5': 'https://mat.tepper.cmu.edu/COLOR/instances/queen5_5.col',  # 25 vertices - small
    'queen8_8': 'https://mat.tepper.cmu.edu/COLOR/instances/queen8_8.col',  # 64 vertices - medium
    'queen11_11': 'https://mat.tepper.cmu.edu/COLOR/instances/queen11_11.col',  # 121 vertices - medium
    'miles250': 'https://mat.tepper.cmu.edu/COLOR/instances/miles250.col',  # 128 vertices - medium
    'le450_15a': 'https://mat.tepper.cmu.edu/COLOR/instances/le450_15a.col',  # 450 vertices - large
}

print("Downloading datasets...")
for name, url in datasets.items():
    filepath = f'datasets/{name}.col'
    if not os.path.exists(filepath):
        try:
            print(f"Downloading {name}...")
            urllib.request.urlretrieve(url, filepath)
            print(f"  Saved to {filepath}")
        except Exception as e:
            print(f"  Error downloading {name}: {e}")
    else:
        print(f"{name} already exists")

print("\nDataset download complete!")

## 8. Test with Small Graph (myciel3 - 11 vertices)

**Dataset**: myciel3.col
- **Vertices**: 11
- **Edges**: 20
- **Optimal chromatic number**: 4
- **Source**: https://mat.tepper.cmu.edu/COLOR/instances/myciel3.col

In [ ]:
# Load small graph
print("Loading myciel3.col...")
graph_small = load_dimacs_graph('datasets/myciel3.col')
print(f"Graph loaded: {graph_small}")
print(f"Max degree: {graph_small.max_degree()}")
print(f"Chromatic upper bound: {graph_small.chromatic_upper_bound()}")

# Visualize the graph
greedy_sol = greedy_coloring(graph_small)
visualize_graph_coloring(graph_small, greedy_sol, "Greedy Coloring - myciel3")

### 8.1 Experiment 1: Tournament Selection + Uniform Crossover + Random Mutation

In [ ]:
print("\n=== Experiment 1: Tournament + Uniform + Random ===")
ga1 = GeneticAlgorithm(
    graph_small,
    population_size=50,
    mutation_rate=0.1,
    crossover_rate=0.8,
    elitism_size=2,
    max_generations=500,
    selection_method='tournament',
    crossover_method='uniform',
    mutation_method='random'
)

start_time = time.time()
solution1, history1 = ga1.evolve(verbose=True)
time1 = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_small, solution1)
num_colors1 = count_colors(solution1)
print(f"\nResult: Colors={num_colors1}, Valid={is_valid}, Conflicts={conflicts}, Time={time1:.2f}s")
print(f"Solution: {solution1}")

plot_fitness_evolution(history1, "Experiment 1 - Fitness Evolution")
visualize_graph_coloring(graph_small, solution1, "Experiment 1 - Best Solution")

### 8.2 Experiment 2: Tournament Selection + Uniform Crossover + Smart Mutation

In [ ]:
print("\n=== Experiment 2: Tournament + Uniform + Smart ===")
ga2 = GeneticAlgorithm(
    graph_small,
    population_size=50,
    mutation_rate=0.1,
    crossover_rate=0.8,
    elitism_size=2,
    max_generations=500,
    selection_method='tournament',
    crossover_method='uniform',
    mutation_method='smart'
)

start_time = time.time()
solution2, history2 = ga2.evolve(verbose=True)
time2 = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_small, solution2)
num_colors2 = count_colors(solution2)
print(f"\nResult: Colors={num_colors2}, Valid={is_valid}, Conflicts={conflicts}, Time={time2:.2f}s")

plot_fitness_evolution(history2, "Experiment 2 - Fitness Evolution")

### 8.3 Experiment 3: Tournament + Single Point Crossover + Smart Mutation

In [ ]:
print("\n=== Experiment 3: Tournament + Single Point + Smart ===")
ga3 = GeneticAlgorithm(
    graph_small,
    population_size=50,
    mutation_rate=0.1,
    crossover_rate=0.8,
    elitism_size=2,
    max_generations=500,
    selection_method='tournament',
    crossover_method='single_point',
    mutation_method='smart'
)

start_time = time.time()
solution3, history3 = ga3.evolve(verbose=True)
time3 = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_small, solution3)
num_colors3 = count_colors(solution3)
print(f"\nResult: Colors={num_colors3}, Valid={is_valid}, Conflicts={conflicts}, Time={time3:.2f}s")

plot_fitness_evolution(history3, "Experiment 3 - Fitness Evolution")

### 8.4 Experiment 4: Roulette Selection + Uniform Crossover + Smart Mutation

In [ ]:
print("\n=== Experiment 4: Roulette + Uniform + Smart ===")
ga4 = GeneticAlgorithm(
    graph_small,
    population_size=50,
    mutation_rate=0.1,
    crossover_rate=0.8,
    elitism_size=2,
    max_generations=500,
    selection_method='roulette',
    crossover_method='uniform',
    mutation_method='smart'
)

start_time = time.time()
solution4, history4 = ga4.evolve(verbose=True)
time4 = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_small, solution4)
num_colors4 = count_colors(solution4)
print(f"\nResult: Colors={num_colors4}, Valid={is_valid}, Conflicts={conflicts}, Time={time4:.2f}s")

plot_fitness_evolution(history4, "Experiment 4 - Fitness Evolution")

### 8.5 Experiment 5: High Mutation Rate

In [ ]:
print("\n=== Experiment 5: High Mutation Rate (0.3) ===")
ga5 = GeneticAlgorithm(
    graph_small,
    population_size=50,
    mutation_rate=0.3,
    crossover_rate=0.8,
    elitism_size=2,
    max_generations=500,
    selection_method='tournament',
    crossover_method='uniform',
    mutation_method='smart'
)

start_time = time.time()
solution5, history5 = ga5.evolve(verbose=True)
time5 = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_small, solution5)
num_colors5 = count_colors(solution5)
print(f"\nResult: Colors={num_colors5}, Valid={is_valid}, Conflicts={conflicts}, Time={time5:.2f}s")

plot_fitness_evolution(history5, "Experiment 5 - Fitness Evolution")

### 8.6 Experiment 6: Large Population

In [ ]:
print("\n=== Experiment 6: Large Population (100) ===")
ga6 = GeneticAlgorithm(
    graph_small,
    population_size=100,
    mutation_rate=0.1,
    crossover_rate=0.8,
    elitism_size=5,
    max_generations=500,
    selection_method='tournament',
    crossover_method='uniform',
    mutation_method='smart'
)

start_time = time.time()
solution6, history6 = ga6.evolve(verbose=True)
time6 = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_small, solution6)
num_colors6 = count_colors(solution6)
print(f"\nResult: Colors={num_colors6}, Valid={is_valid}, Conflicts={conflicts}, Time={time6:.2f}s")

plot_fitness_evolution(history6, "Experiment 6 - Fitness Evolution")

### 8.7 Simulated Annealing on Small Graph

In [ ]:
print("\n=== Simulated Annealing ===")
sa = SimulatedAnnealing(
    graph_small,
    initial_temp=100.0,
    cooling_rate=0.95,
    max_iterations=5000
)

start_time = time.time()
solution_sa, history_sa = sa.optimize(verbose=True)
time_sa = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_small, solution_sa)
num_colors_sa = count_colors(solution_sa)
print(f"\nResult: Colors={num_colors_sa}, Valid={is_valid}, Conflicts={conflicts}, Time={time_sa:.2f}s")

plot_fitness_evolution(history_sa, "Simulated Annealing - Fitness Evolution")

### 8.8 Tabu Search on Small Graph

In [ ]:
print("\n=== Tabu Search ===")
ts = TabuSearch(
    graph_small,
    tabu_tenure=10,
    max_iterations=5000
)

start_time = time.time()
solution_ts, history_ts = ts.optimize(verbose=True)
time_ts = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_small, solution_ts)
num_colors_ts = count_colors(solution_ts)
print(f"\nResult: Colors={num_colors_ts}, Valid={is_valid}, Conflicts={conflicts}, Time={time_ts:.2f}s")

plot_fitness_evolution(history_ts, "Tabu Search - Fitness Evolution")

### 8.9 Summary and Comparison - Small Graph

In [ ]:
# Create summary table
results_small = {
    'Experiment': ['GA-Exp1', 'GA-Exp2', 'GA-Exp3', 'GA-Exp4', 'GA-Exp5', 'GA-Exp6', 'SA', 'TS'],
    'Colors': [num_colors1, num_colors2, num_colors3, num_colors4, num_colors5, num_colors6, num_colors_sa, num_colors_ts],
    'Time(s)': [time1, time2, time3, time4, time5, time6, time_sa, time_ts],
    'Iterations': [len(history1), len(history2), len(history3), len(history4), len(history5), len(history6), len(history_sa), len(history_ts)],
    'Configuration': [
        'Tournament+Uniform+Random',
        'Tournament+Uniform+Smart',
        'Tournament+SinglePt+Smart',
        'Roulette+Uniform+Smart',
        'HighMutation(0.3)',
        'LargePop(100)',
        'SA(T=100,α=0.95)',
        'TS(tenure=10)'
    ]
}

df_small = pd.DataFrame(results_small)
print("\n=== Results Summary - myciel3 (11 vertices) ===")
print(df_small.to_string(index=False))

# Find best configuration
best_idx = df_small['Colors'].idxmin()
print(f"\n*** Best Configuration: {df_small.loc[best_idx, 'Experiment']} with {df_small.loc[best_idx, 'Colors']} colors ***")
print(f"    Optimal chromatic number for myciel3: 4")

## 9. Test with Medium Graph (queen8_8 - 64 vertices)

**Dataset**: queen8_8.col
- **Vertices**: 64
- **Edges**: 728
- **Optimal chromatic number**: 9
- **Source**: https://mat.tepper.cmu.edu/COLOR/instances/queen8_8.col
- **Description**: Represents queen's graph on 8×8 chessboard

In [ ]:
# Load medium graph
print("Loading queen8_8.col...")
graph_medium = load_dimacs_graph('datasets/queen8_8.col')
print(f"Graph loaded: {graph_medium}")
print(f"Max degree: {graph_medium.max_degree()}")
print(f"Chromatic upper bound: {graph_medium.chromatic_upper_bound()}")

### 9.1 Best GA Configuration from Small Graph Test

In [ ]:
print("\n=== GA Best Config on Medium Graph ===")
ga_medium = GeneticAlgorithm(
    graph_medium,
    population_size=100,
    mutation_rate=0.1,
    crossover_rate=0.8,
    elitism_size=5,
    max_generations=1000,
    selection_method='tournament',
    crossover_method='uniform',
    mutation_method='smart'
)

start_time = time.time()
solution_ga_med, history_ga_med = ga_medium.evolve(verbose=True)
time_ga_med = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_medium, solution_ga_med)
num_colors_ga_med = count_colors(solution_ga_med)
print(f"\nGA Result: Colors={num_colors_ga_med}, Valid={is_valid}, Conflicts={conflicts}, Time={time_ga_med:.2f}s")

plot_fitness_evolution(history_ga_med, "GA - Medium Graph - Fitness Evolution")

### 9.2 Simulated Annealing on Medium Graph

In [ ]:
print("\n=== Simulated Annealing on Medium Graph ===")
sa_medium = SimulatedAnnealing(
    graph_medium,
    initial_temp=200.0,
    cooling_rate=0.95,
    max_iterations=10000
)

start_time = time.time()
solution_sa_med, history_sa_med = sa_medium.optimize(verbose=True)
time_sa_med = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_medium, solution_sa_med)
num_colors_sa_med = count_colors(solution_sa_med)
print(f"\nSA Result: Colors={num_colors_sa_med}, Valid={is_valid}, Conflicts={conflicts}, Time={time_sa_med:.2f}s")

plot_fitness_evolution(history_sa_med, "SA - Medium Graph - Fitness Evolution")

### 9.3 Tabu Search on Medium Graph

In [ ]:
print("\n=== Tabu Search on Medium Graph ===")
ts_medium = TabuSearch(
    graph_medium,
    tabu_tenure=15,
    max_iterations=10000
)

start_time = time.time()
solution_ts_med, history_ts_med = ts_medium.optimize(verbose=True)
time_ts_med = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_medium, solution_ts_med)
num_colors_ts_med = count_colors(solution_ts_med)
print(f"\nTS Result: Colors={num_colors_ts_med}, Valid={is_valid}, Conflicts={conflicts}, Time={time_ts_med:.2f}s")

plot_fitness_evolution(history_ts_med, "TS - Medium Graph - Fitness Evolution")

### 9.4 Summary - Medium Graph

In [ ]:
results_medium_comp = {
    'GA': {'colors': num_colors_ga_med, 'time': time_ga_med, 'iterations': len(history_ga_med)},
    'SA': {'colors': num_colors_sa_med, 'time': time_sa_med, 'iterations': len(history_sa_med)},
    'TS': {'colors': num_colors_ts_med, 'time': time_ts_med, 'iterations': len(history_ts_med)}
}

plot_algorithm_comparison(results_medium_comp)

print("\n=== Results Summary - queen8_8 (64 vertices) ===")
print(f"GA: {num_colors_ga_med} colors in {time_ga_med:.2f}s ({len(history_ga_med)} iterations)")
print(f"SA: {num_colors_sa_med} colors in {time_sa_med:.2f}s ({len(history_sa_med)} iterations)")
print(f"TS: {num_colors_ts_med} colors in {time_ts_med:.2f}s ({len(history_ts_med)} iterations)")
print(f"\nOptimal chromatic number for queen8_8: 9")

## 10. Test with Large Graph (le450_15a - 450 vertices)

**Dataset**: le450_15a.col
- **Vertices**: 450
- **Edges**: 8168
- **Optimal chromatic number**: 15
- **Source**: https://mat.tepper.cmu.edu/COLOR/instances/le450_15a.col
- **Description**: Leighton graph

In [ ]:
# Load large graph
print("Loading le450_15a.col...")
graph_large = load_dimacs_graph('datasets/le450_15a.col')
print(f"Graph loaded: {graph_large}")
print(f"Max degree: {graph_large.max_degree()}")
print(f"Chromatic upper bound: {graph_large.chromatic_upper_bound()}")

### 10.1 GA on Large Graph

In [ ]:
print("\n=== GA on Large Graph ===")
ga_large = GeneticAlgorithm(
    graph_large,
    population_size=150,
    mutation_rate=0.1,
    crossover_rate=0.8,
    elitism_size=10,
    max_generations=2000,
    selection_method='tournament',
    crossover_method='uniform',
    mutation_method='smart'
)

start_time = time.time()
solution_ga_large, history_ga_large = ga_large.evolve(verbose=True)
time_ga_large = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_large, solution_ga_large)
num_colors_ga_large = count_colors(solution_ga_large)
print(f"\nGA Result: Colors={num_colors_ga_large}, Valid={is_valid}, Conflicts={conflicts}, Time={time_ga_large:.2f}s")

plot_fitness_evolution(history_ga_large, "GA - Large Graph - Fitness Evolution")

### 10.2 Simulated Annealing on Large Graph

In [ ]:
print("\n=== Simulated Annealing on Large Graph ===")
sa_large = SimulatedAnnealing(
    graph_large,
    initial_temp=500.0,
    cooling_rate=0.98,
    max_iterations=20000
)

start_time = time.time()
solution_sa_large, history_sa_large = sa_large.optimize(verbose=True)
time_sa_large = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_large, solution_sa_large)
num_colors_sa_large = count_colors(solution_sa_large)
print(f"\nSA Result: Colors={num_colors_sa_large}, Valid={is_valid}, Conflicts={conflicts}, Time={time_sa_large:.2f}s")

plot_fitness_evolution(history_sa_large, "SA - Large Graph - Fitness Evolution")

### 10.3 Tabu Search on Large Graph

In [ ]:
print("\n=== Tabu Search on Large Graph ===")
ts_large = TabuSearch(
    graph_large,
    tabu_tenure=20,
    max_iterations=20000
)

start_time = time.time()
solution_ts_large, history_ts_large = ts_large.optimize(verbose=True)
time_ts_large = time.time() - start_time

is_valid, conflicts = validate_coloring(graph_large, solution_ts_large)
num_colors_ts_large = count_colors(solution_ts_large)
print(f"\nTS Result: Colors={num_colors_ts_large}, Valid={is_valid}, Conflicts={conflicts}, Time={time_ts_large:.2f}s")

plot_fitness_evolution(history_ts_large, "TS - Large Graph - Fitness Evolution")

### 10.4 Summary - Large Graph

In [ ]:
results_large_comp = {
    'GA': {'colors': num_colors_ga_large, 'time': time_ga_large, 'iterations': len(history_ga_large)},
    'SA': {'colors': num_colors_sa_large, 'time': time_sa_large, 'iterations': len(history_sa_large)},
    'TS': {'colors': num_colors_ts_large, 'time': time_ts_large, 'iterations': len(history_ts_large)}
}

plot_algorithm_comparison(results_large_comp)

print("\n=== Results Summary - le450_15a (450 vertices) ===")
print(f"GA: {num_colors_ga_large} colors in {time_ga_large:.2f}s ({len(history_ga_large)} iterations)")
print(f"SA: {num_colors_sa_large} colors in {time_sa_large:.2f}s ({len(history_sa_large)} iterations)")
print(f"TS: {num_colors_ts_large} colors in {time_ts_large:.2f}s ({len(history_ts_large)} iterations)")
print(f"\nOptimal chromatic number for le450_15a: 15")

## 11. Final Comparison - All Graphs and Algorithms

In [ ]:
# Create comprehensive comparison
all_results = pd.DataFrame({
    'Graph': ['myciel3 (11v)', 'myciel3 (11v)', 'myciel3 (11v)', 
              'queen8_8 (64v)', 'queen8_8 (64v)', 'queen8_8 (64v)',
              'le450_15a (450v)', 'le450_15a (450v)', 'le450_15a (450v)'],
    'Algorithm': ['GA', 'SA', 'TS', 'GA', 'SA', 'TS', 'GA', 'SA', 'TS'],
    'Colors': [num_colors6, num_colors_sa, num_colors_ts,
               num_colors_ga_med, num_colors_sa_med, num_colors_ts_med,
               num_colors_ga_large, num_colors_sa_large, num_colors_ts_large],
    'Time(s)': [time6, time_sa, time_ts,
                time_ga_med, time_sa_med, time_ts_med,
                time_ga_large, time_sa_large, time_ts_large],
    'Iterations': [len(history6), len(history_sa), len(history_ts),
                   len(history_ga_med), len(history_sa_med), len(history_ts_med),
                   len(history_ga_large), len(history_sa_large), len(history_ts_large)],
    'Optimal': [4, 4, 4, 9, 9, 9, 15, 15, 15]
})

print("\n" + "="*80)
print(" "*20 + "FINAL RESULTS COMPARISON")
print("="*80)
print(all_results.to_string(index=False))
print("="*80)

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, graph_name in enumerate(['myciel3 (11v)', 'queen8_8 (64v)', 'le450_15a (450v)']):
    data = all_results[all_results['Graph'] == graph_name]
    
    x = np.arange(len(data))
    width = 0.35
    
    ax = axes[i]
    ax2 = ax.twinx()
    
    bars1 = ax.bar(x - width/2, data['Colors'], width, label='Colors', color='steelblue')
    bars2 = ax2.bar(x + width/2, data['Time(s)'], width, label='Time(s)', color='coral')
    
    ax.set_xlabel('Algorithm')
    ax.set_ylabel('Number of Colors', color='steelblue')
    ax2.set_ylabel('Time (seconds)', color='coral')
    ax.set_title(graph_name)
    ax.set_xticks(x)
    ax.set_xticklabels(data['Algorithm'])
    ax.tick_params(axis='y', labelcolor='steelblue')
    ax2.tick_params(axis='y', labelcolor='coral')
    
    # Add optimal line
    optimal = data['Optimal'].iloc[0]
    ax.axhline(y=optimal, color='green', linestyle='--', label=f'Optimal={optimal}')
    ax.legend(loc='upper left')

plt.tight_layout()
plt.show()

## 12. Conclusions

### Chromosome Representation
- Direct encoding: `chromosome[i]` = color of vertex `i`
- Simple and effective for graph coloring
- Allows easy validation and mutation

### Selection Methods Implemented
1. **Tournament Selection**: Better for exploration, consistently good results
2. **Roulette Wheel Selection**: Faster convergence but may get stuck in local optima

### Crossover Methods Implemented
1. **Uniform Crossover**: Better mixing of parent genes, more diversity
2. **Single Point Crossover**: Preserves larger blocks of genes, faster computation

### Mutation Methods Implemented
1. **Random Mutation**: Simple, maintains diversity
2. **Smart Mutation**: Targets conflicting vertices, faster convergence to valid solutions

### Algorithm Comparison
- **Genetic Algorithm**: Best balance of quality and scalability
- **Simulated Annealing**: Fast for small graphs, struggles with larger instances
- **Tabu Search**: Good exploration but slower convergence

### Population Size and Stationary State
- Larger populations (100-150) give better results but take more time
- Convergence detected when no improvement for 50+ generations
- Elitism (keeping best 2-10 individuals) crucial for maintaining good solutions